In [1]:
from pyspark import SparkConf, SparkContext
# SparkConf : 환경 설정
# SparkContext : 실제 개발 프로그램. 드라이브
#conf = SparkConf().setMaster("yarn") # yarn으로 하면 자동으로 하둡을 사용하게 된다
conf = SparkConf().setMaster("local").setAppName("transformations_actions")
sc = SparkContext(conf=conf) # sc 변수명 바꾸면 안된다. 실무에서는 스파크 서브밋 이라는 것을 사용한다. 그때 필수

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/25 21:40:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# RDD 기반의 데이터 처리

In [3]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

마블 영화 중 관객수가 500만 이상인 영화 가져오기

In [5]:
# Case 1 : Join 먼저, filter 나중에 (좋지 않은 방법)
movie_att = movies_rdd.join(attendances_rdd)
movie_att.collect()

[(2, (('슈퍼맨', 'DC'), (2182227, 'KR'))),
 (4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (3, (('배트맨', 'DC'), (4226242, 'KR'))),
 (5, (('아이언맨', '마블'), (4300365, 'KR')))]

In [7]:
movie_att.filter(lambda x : x[1][0][1]=='마블' and x[1][1][0]>=5000000).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [8]:
# Case 2 : Filter 먼저, Join 나중
filtered_movies = movies_rdd.filter(lambda x : x[1][1] =='마블')
filtered_att = attendances_rdd.filter(lambda x : x[1][0] >= 5000000)
filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [9]:
sc.stop()

# Spark SQL 사용하기

## Spark Session 생성
- SparkContext에 해당하며, 새로운 스파크 어플리케이션을 생성

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()
spark

In [12]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

movie_schema = ["id", "name", "company", "year", "month", "day"]

# 데이터 프레임 생성
- `inferSchema=True`로 설정하면 데이터의 타입을 스파크가 자동으로 결정(default)

In [14]:
df = spark.createDataFrame(data=movies, schema=movie_schema)
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [15]:
df # 이게 곧 RDD. RDD의 역할을 한다!

DataFrame[id: bigint, name: string, company: string, year: bigint, month: bigint, day: bigint]

In [16]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



# 데이터 프레임에 SQL 사용하기
- 데이터 프레임을 temporary view(In Memory Table)에 등록해야 SparkSQL을 사용할 수 있다!

In [17]:
df.createOrReplaceTempView("movies") # movies라는 임시 테이블 생성

In [18]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- company: string (nullable = true)
 |-- year: long (nullable = true)
 |-- month: long (nullable = true)
 |-- day: long (nullable = true)



In [19]:
query = """
SELECT name
FROM movies
"""

spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



# 2010년 이후에 개방한 영화를 조회해 보세요

In [22]:
query = """
select * from movies where year>=2010
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [ ]:
# 제목이 --맨으로 끝나는 영화정보 조회

In [26]:
query = """
select * from movies where name like "%맨"
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [28]:
# 개봉월이 4~8월 사이인 영화 정보 조회
query = """
select * from movies where month between 4 and 8
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [31]:
# 회사 이름이"마"로 시작하거나 "니"로 끝나는 영화 중 2010년 이후로 개봉한 영화
query = """
select * from 
(select * from movies where company like "마%" or company like "%니") as t1
where t1.year >2010
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [33]:
# 개봉 년도 오름차순 정렬
query = """
select *
from movies
order by year
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [34]:
# 회사 별 영화 개수 확인하기
query = """
select company, count(*) as movie_count
from movies
group by company
"""
spark.sql(query).show()

+-------+-----------+
|company|movie_count|
+-------+-----------+
| 디즈니|          1|
|   마블|          2|
|     DC|          2|
+-------+-----------+



In [35]:
spark.stop()